In [1]:
import os
import pandas as pd
import numpy as np

file = "Resources/purchase_data.csv"
df = pd.read_csv(file)
df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [2]:
#  Total number of players
num_players = [{"Total Players": df['SN'].nunique()}]
tot_players = pd.DataFrame(num_players)
#  Unique Items
un_items = df['Item ID'].nunique()
# print("Unique Items",un_items)
# Average Purchase Price
ave_price = round(df['Price'].mean(),2)
# print("Average Price",ave_price)
# Number of Purchases
num_purch = len(df)
# print("Number of Purchases",num_purch)
tot_rev = df['Price'].sum()
# print("Total Revenue", tot_rev)
sum_cols = pd.DataFrame({"Number of Unique Items": [un_items], 
                         "Average Price": [ave_price], 
                         "Number of Purchases": [num_purch], 
                         "Total Revenue": [tot_rev]},)
sum_cols["Average Price"] = sum_cols["Average Price"].map("${:.2f}".format)
sum_cols["Total Revenue"] = sum_cols["Total Revenue"].map("${:.2f}".format)


In [3]:
df_gencount = df.groupby('Gender').agg({'SN':'nunique'})
df_gencount['Percentage of Players'] = df_gencount['SN']*100/num_players[0].get("Total Players")
df_gencount['Percentage of Players'] = df_gencount['Percentage of Players'].map("{:.2f}%".format)
df_gencount = df_gencount.rename(columns={'SN':"Total Count"})

In [4]:
df_gender = df.groupby('Gender').agg({'Purchase ID': 'count','Price':['mean','sum']})
df_gender.columns = df_gender.columns.map(' '.join)
df_gender['Avg Total Purchase per Person'] = df_gender['Price sum']/df_gencount.get("Total Count")
df_gender = df_gender.rename(columns={"Purchase ID count":"Purchase Count",
                                      "Price mean":"Average Purchase Price",
                                     "Price sum":"Total Purchase Value"})
df_gender['Average Purchase Price'] = df_gender['Average Purchase Price'].map("${:.2f}".format)
df_gender['Total Purchase Value'] = df_gender['Total Purchase Value'].map("${:.2f}".format)
df_gender['Avg Total Purchase per Person'] = df_gender['Avg Total Purchase per Person'].map("${:.2f}".format)

In [5]:
bins = [0,9,14,19,24,29,34,39,200]
bin_names = ['<10', '10-14','15-19','20-24','25-29','30-34','35-39','40+']
df["Age Group"] = pd.cut(df["Age"], bins, labels=bin_names)
df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price,Age Group
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53,20-24
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56,40+
2,2,Ithergue48,24,Male,92,Final Critic,4.88,20-24
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27,20-24
4,4,Iskosia90,23,Male,131,Fury,1.44,20-24


In [6]:
age_demo = df.groupby('Age Group').agg({'SN': 'nunique'})
age_demo = age_demo.rename(columns={'SN':'Total Count'})
age_demo['Percentage of Players'] = age_demo['Total Count']*100/num_players[0].get("Total Players")
age_demo['Percentage of Players'] = age_demo['Percentage of Players'].map("{:.2f}%".format)

In [7]:
df_age = df.groupby('Age Group').agg({'SN': 'nunique','Price':['mean','sum']})
df_age.columns = df_age.columns.map(' '.join)
df_age['Avg Total Purchase per Person'] = df_age['Price sum']/df_age['SN nunique']
df_age = df_age.rename(columns={'SN nunique':'Purchase Count',
                                'Price mean':'Average Purchase Price',
                                'Price sum':'Total Purchase Value'})
df_age['Average Purchase Price'] = df_age['Average Purchase Price'].map("${:.2f}".format)
df_age['Total Purchase Value'] = df_age['Total Purchase Value'].map("${:.2f}".format)
df_age['Avg Total Purchase per Person'] = df_age['Avg Total Purchase per Person'].map("${:.2f}".format)

In [8]:
top_spenders = df.groupby('SN').agg({'Price':['sum', 'mean'],'Purchase ID': 'count'})
top_spenders.columns = top_spenders.columns.map(' '.join)
top_spenders = top_spenders[['Purchase ID count', "Price mean", "Price sum"]]
top_spenders = top_spenders.rename(columns={'Purchase ID count':'Purchase count',
                                           "Price mean":"Average Purchase Price",
                                            "Price sum":"Total Purchase Value"})
top_spenders['Average Purchase Price'] = top_spenders['Average Purchase Price'].map("${:.2f}".format)

# when this line is included, it screws everything up.
# top_spenders["Total Purchase Value"] = top_spenders['Total Purchase Value'].map("${:.2f}".format)

top_spenders = top_spenders.sort_values(by='Total Purchase Value', ascending=False)
top_spenders_sum = top_spenders.head()
top_spenders_sum["Total Purchase Value"] = top_spenders_sum['Total Purchase Value'].map("${:.2f}".format)
top_spenders_sum

C:\Users\jarre\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Purchase count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [9]:
pop_items = df.groupby(['Item ID','Item Name']).agg({'Price':['sum', 'mean'],'Purchase ID': 'count'})
pop_items.columns = pop_items.columns.map(' '.join)
pop_items = pop_items[['Purchase ID count', 'Price mean', 'Price sum']]
pop_items = pop_items.rename(columns={'Purchase ID count':'Purchase Count',
                                     'Price mean':'Item Price',
                                     'Price sum':'Total Purchase Value'})
pop_items['Item Price'] = pop_items['Item Price'].map("${:.2f}".format)

# same issue as above
# pop_items['Total Purchase Value'] = pop_items['Total Purchase Value'].map("${:.2f}".format)

pop_items_pop = pop_items.sort_values(by='Purchase Count', ascending=False).head()
pop_items_pop['Total Purchase Value'] = pop_items_pop['Total Purchase Value'].map("${:.2f}".format)
pop_items_prof = pop_items.sort_values(by='Total Purchase Value', ascending=False).head()
pop_items_prof['Total Purchase Value'] = pop_items_prof['Total Purchase Value'].map("${:.2f}".format)

# Summary Starts Here

In [10]:
tot_players

,Total Players
0,576


In [11]:
sum_cols

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,$2379.77


In [12]:
df_gencount

,Total Count,Percentage of Players
Gender,,
Female,81,14.06%
Male,484,84.03%
Other / Non-Disclosed,11,1.91%


In [13]:
df_gender

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,$1967.64,$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [14]:
age_demo

,Total Count,Percentage of Players
Age Group,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


In [15]:
df_age

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Group,,,,
<10,17,$3.35,$77.13,$4.54
10-14,22,$2.96,$82.78,$3.76
15-19,107,$3.04,$412.89,$3.86
20-24,258,$3.05,$1114.06,$4.32
25-29,77,$2.90,$293.00,$3.81
30-34,52,$2.93,$214.00,$4.12
35-39,31,$3.60,$147.67,$4.76
40+,12,$2.94,$38.24,$3.19


In [16]:
top_spenders_sum

,Purchase count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [17]:
pop_items_pop

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


In [18]:
pop_items_prof

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80


# Observations
1. 19-24-year-olds are the largest age group of spenders. They also spend significantly more than the next age group older. It would be worth pursing the current 19-24s as the age to improve the numbers of the next bracket.
2. Similarly, while self-identified male customer sales vastly outnumber the self-idendtified female sales, the latter completes, on average, 10% more expensive transactions.
3. This is not a very large gaming company.